<a href="https://colab.research.google.com/github/sudharsan99-hacker/CNN-GRU-Based-Hybrid-Architecture-for-Video-Classification-of-Birds/blob/main/CNN_GRU_Based_Hybrid_Architecture_for_Video_Classification_of_Birds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
from imutils import paths
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

!pip install git+https://github.com/tensorflow/docs

from tensorflow_docs.vis import embed

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-pp8jvk99
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-pp8jvk99
  Resolved https://github.com/tensorflow/docs to commit a45eae2e4cd45ed2593793d1cdb4ed6990a807e9
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-docs: filename=tensorflow_docs-2024.7.15.51478-py3-none-any.whl size=182586 sha256=767d2eb786ef6bfc9e73cc55ce52e885a51df42440df69081a2f55603b38085e
  Stored in directory: /tmp/pip-ephem-wheel-cache-8tceg9nm/wheels/86/0f/1e/3b62293c8ffd0fd5a49508e6871cdb7554abe9c62afd35ec53
Successfully built tensorflow-docs


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
dataset_path = os.listdir("/content/drive/MyDrive/Dataset/Training")

rooms = []
for item in dataset_path:
    all_rooms = os.listdir('/content/drive/MyDrive/Dataset/Training' + '/' +item)

    for room in all_rooms:
        rooms.append((item, str('/content/drive/MyDrive/Dataset/Training' + '/' +item) + '/' + room))

train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])

df = train_df.loc[:,['video_name','tag']]
df.to_csv('train.csv')

In [ ]:
dataset_path = os.listdir('/content/drive/MyDrive/Dataset/Testing')

room_types = os.listdir('/content/drive/MyDrive/Dataset/Testing')

rooms = []

for item in dataset_path:
    all_rooms = os.listdir('/content/drive/MyDrive/Dataset/Testing' + '/' +item)

    for room in all_rooms:
        rooms.append((item, str('/content/drive/MyDrive/Dataset/Testing' + '/' +item) + '/' + room))

test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])

df = test_df.loc[:,['video_name','tag']]
df.to_csv('test.csv')

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

Total videos for training: 50
Total videos for testing: 15


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 5

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [ ]:
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [ ]:
from keras.applications.resnet import preprocess_input
def build_feature_extractor():
    feature_extractor = keras.applications.resnet.ResNet50(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_inpu = preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_inpu(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['Allen_Hummingbird', 'Cassin_Finch', 'Lark_Sparrow', 'Laughing_Gull']


array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()


    labels = df["tag"].values


    labels = label_processor(labels[..., None]).numpy()


    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")


    for idx, path in enumerate(video_paths):

        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]


        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )


        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [18]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    # x = keras.layers.GRU(8)(x)
    # x = keras.layers.Dropout(0.4)(x)
    # x = keras.layers.Dense(8, activation="relu")(x)
    # output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    x = keras.layers.LSTM(100, return_sequences=False)(frame_features_input, mask=mask_input)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(100, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    lstm_model = keras.Model([frame_features_input, mask_input], output)

    lstm_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return lstm_model

EPOCHS = 30

def run_experiment():
    filepath = "./tmp/video_classifier.weights.h5"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
1/2 ━━━━━━━━━━━━━━━━━━━━ 3s 4s/step - accuracy: 0.3750 - loss: 1.2663
Epoch 1: val_loss improved from inf to 2.55330, saving model to ./tmp/video_classifier.weights.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 458ms/step - accuracy: 0.3917 - loss: 1.2364 - val_accuracy: 0.0667 - val_loss: 2.5533
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.6424 - loss: 0.8876 
Epoch 2: val_loss did not improve from 2.55330
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step - accuracy: 0.6378 - loss: 0.8899 - val_accuracy: 0.0667 - val_loss: 3.0829
Epoch 3/30
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step - accuracy: 0.9375 - loss: 0.4990
Epoch 3: val_loss did not improve from 2.55330
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.9411 - loss: 0.4893 - val_accuracy: 0.0667 - val_loss: 3.1740
Epoch 4/30
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 1.0000 - loss: 0.3858
Epoch 4: val_loss did not improve from 2.55330
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.9810 - loss: 0.4073 - val_accurac